In [172]:
import pandas as pd
pd.set_option("display.max_colwidth", None)
toxic_comments = pd.read_csv("/Users/ekaterinakastaleva/russian_comments_from_2ch_pikabu.csv")

In [173]:
import numpy as np
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ekaterinakastaleva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [174]:
toxic_comments.head()

,comment,toxic,translated
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"Camels, for what? Morons, bl ..."
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0,"Ukrainians, this is an outlet vent zatyukanogo Russians, they say, out, and Ukrainians worse. If there were no Ukrainians, jelly would have invented them."
2,Собаке - собачья смерть\n,1.0,Dog - Dog Death
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0,"Refresh the page, moron. This is also not an insult, but a proven fact - I will not write a non-moron to myself in the plural. Or do we believe in you - are you and your imaginary friends?"
4,"тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n",1.0,did not convince you of the 6-page pdf that Skripale was poisoned by Russia? Are you trying to analyze and think? Padded jacket or what?)


In [175]:
type(toxic_comments)

pandas.core.frame.DataFrame

In [176]:
toxic_comments.columns=[ "comment", "toxic","translated"]

In [177]:
#I dont need the translated-column, because of the bad translation
toxic_comments=toxic_comments[["comment","toxic"]]
toxic_comments.columns

Index(['comment', 'toxic'], dtype='object')

In [178]:
#lowercase all words
toxic_comments['comment'] = toxic_comments["comment"].str.lower()

In [179]:
type(toxic_comments)

pandas.core.frame.DataFrame

In [180]:
print(toxic_comments[5:10])

                                                                                                                                                                                                                                                                                                      comment  \
5                                                                                                                                                                                                    для каких стан является эталоном современная система здравоохранения рф? для зимбабве? ты тупой? хохлы\n   
6  в шапке были ссылки на инфу по текущему фильму марвел. эти ссылки были заменены на фразу репортим брипидора, игнорируем его посты. если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.\n   
7                                                                                    

In [142]:
#toxic_comments.to_frame()

In [181]:
#remove all links
def remove_links(column):
    return re.sub('((www.[^s]+)|(https?://[^s]+))', ' ',column)

In [182]:
toxic_comments['comment'] = toxic_comments['comment'].apply(lambda column: remove_links(column))

In [183]:
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

In [184]:
print(toxic_comments['comment'][5])
print(russian_stopwords)
len(russian_stopwords)

для каких стан является эталоном современная система здравоохранения рф? для зимбабве? ты тупой? хохлы

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', '

151

In [185]:
def prep(column, threshold = 1):
    threshold = threshold
    tokens = mystem.lemmatize(column.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
             and token !=" " \
             and token.strip() not in punctuation]
    column = " ".join([word for word in tokens if len(word) > threshold])
    return re.sub('[0-9]+', '', column)

In [186]:
toxic_comments['comment'] = toxic_comments["comment"].apply(lambda column: prep(column))

In [187]:
print(toxic_comments['comment'][5])

стан являться эталон современный система здравоохранение рф зимбабве тупой хохол


In [188]:
toxic_comments.head()

,comment,toxic
0,верблюд дебил бл ...,1.0
1,хохол это отдушина затюканый россиянин мол вон хохлов плохо хохлов кисель придумывать,1.0
2,собака собачий смерть,1.0
3,страница обновлять дебил это оскорбление доказывать факт дебил множественный число писать верить это твой воображаемый друг,1.0
4,убеждать страничный пдф скрипалый отравлять россия анализировать думать пытаться ватник,1.0


In [189]:
print(toxic_comments['comment'][5:10])

5                                                                                                                                                    стан являться эталон современный система здравоохранение рф зимбабве тупой хохол
6    шапка ссылка инф текущий фильм марвести ссылка заменять фраза репортить брипидор игнорировать пост это недостаточно понимать модератор абсолютный неадекват нужно лишать полномочие борд пробивать абсолютный дно неадекватность
7                                                                                                                                          упад строить технология разворовывать трещина пош тупой китаз мочь ничто нормально сделать
8                                                                                                                                                                                                               ебать разносить шизик
9                                                                               

In [190]:
only_toxic_comments = toxic_comments[toxic_comments["toxic"]==1.0]

In [191]:
print(len(only_toxic_comments))
only_toxic_comments[5:10]

4826


,comment,toxic
5,стан являться эталон современный система здравоохранение рф зимбабве тупой хохол,1.0
7,упад строить технология разворовывать трещина пош тупой китаз мочь ничто нормально сделать,1.0
8,ебать разносить шизик,1.0
9,обсираться сидеть обтекать,1.0
10,писать хуйня дегенерат поцелуй губа поцеловать засос,1.0


In [192]:
toxic_comments['toxic'].value_counts()

0.0    9586
1.0    4826
Name: toxic, dtype: int64

In [193]:
normal_comments = toxic_comments[toxic_comments["toxic"]==0.0]

In [194]:
np.sum(toxic_comments.isnull().any(axis=1))

0

In [195]:
#I have now two classes: normal_comments and only_toxic_comments
normal_comments = normal_comments.iloc[:int(4800)]
print("There are " + str(len(normal_comments)) + " comments with non toxic meaning.")

There are 4800 comments with non toxic meaning.


In [196]:
only_toxic_comments = only_toxic_comments.iloc[:int(4800)]
print("There are " + str(len(only_toxic_comments)) + " comments with toxic meaning.")

There are 4800 comments with toxic meaning.


In [197]:
toxic_dataset = pd.concat([only_toxic_comments, normal_comments])
print(len(toxic_dataset))

9600


In [198]:
toxic_dataset['comment'] = toxic_dataset['comment'].apply(lambda column: numbers(column))

In [199]:
toxic_comments=toxic_dataset.comment

In [201]:
toxic_comments[5:10]

5               стан являться эталон современный система здравоохранение рф зимбабве тупой хохол
7     упад строить технология разворовывать трещина пош тупой китаз мочь ничто нормально сделать
8                                                                          ебать разносить шизик
9                                                                     обсираться сидеть обтекать
10                                          писать хуйня дегенерат поцелуй губа поцеловать засос
Name: comment, dtype: object

In [202]:
#TF-IDF verctorarization
from sklearn.feature_extraction.text import TfidfVectorizer 

In [207]:
vectorizer = TfidfVectorizer(
ngram_range = (1, 3),
use_idf = True,
smooth_idf = False,
norm = None,
decode_error = 'replace',
max_features = 10000,
max_df = 1, #max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".
min_df = 1) #min_df = 5 means "ignore terms that appear in less than 5 documents".

In [208]:
tfidf = vectorizer.fit_transform(toxic_comments).toarray()

In [209]:
tfidf.T.shape

(10000, 9600)

In [210]:
print(tfidf[10:25])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
